<a href="https://colab.research.google.com/github/thesaahilraj/Sarcasm-detector/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

Adding Dataset from Kaggle to Colab

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# %cd /content/gdrive/My Drive/Kaggle
# !kaggle datasets download -d saurabhbagchi/sarcasm-detection-through-nlp
# !unzip \*.zip  && rm *.zip

Mounted at /content/gdrive


In [ ]:
data = '/content/gdrive/MyDrive/Kaggle/Sarcasm_Headlines_Dataset.json'
df = pd.read_json(data, lines=True)
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
# df = df.drop(['article_link'],axis=1)
X = df['headline']
Y = df['is_sarcastic']
print(X.shape),print(Y.shape)

(26709,)
(26709,)


(None, None)

In [ ]:
messages = X.copy()

In [ ]:
messages[1]

"the 'roseanne' revival catches up to our thorny political mood, for better and worse"

In [ ]:
messages.head()

0    former versace store clerk sues over secret 'b...
1    the 'roseanne' revival catches up to our thorn...
2    mom starting to fear son's web series closest ...
3    boehner just wants wife to listen, not come up...
4    j.k. rowling wishes snape happy birthday in th...
Name: headline, dtype: object

**Data Preprocessing** 

In [ ]:
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
snowball = SnowballStemmer("english")
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages[i])
    review = review.lower()
    review = review.split()
    review = [snowball.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages[i])
    review = review.lower()
    corpus.append(review)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import SpatialDropout1D

In [ ]:
### Vocabulary size
voc_size = 20031
onehot_repr=[one_hot(words,voc_size)for words in corpus]

**Embedding Representation**

In [ ]:
sent_length = 40
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ... 13569 15466  8757]
 [    0     0     0 ...  2513 19170  4213]
 [    0     0     0 ... 17254 16607  1247]
 ...
 [    0     0     0 ...  5307 19170  5227]
 [    0     0     0 ... 16316  6599 11077]
 [    0     0     0 ... 13569  9161 16813]]


In [ ]:
len(embedded_docs),Y.shape

(26709, (26709,))

In [ ]:
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [ ]:
X_final.shape,Y_final.shape

((26709, 40), (26709,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, Y_final, test_size=0.25, random_state=42)

In [ ]:
X_train.shape

(20031, 40)

**Model Creation**

In [ ]:
embedding_vector_features = 64

model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(200, dropout=0.2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 40, 64)            1281984   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 40, 64)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 400)               424000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               51328     
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                

**Model Training**

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=32)

Epoch 1/5
626/626 [==============================] - 18s 25ms/step - loss: 0.5291 - accuracy: 0.7083 - val_loss: 0.3500 - val_accuracy: 0.8504
Epoch 2/5
626/626 [==============================] - 14s 23ms/step - loss: 0.2524 - accuracy: 0.8995 - val_loss: 0.3515 - val_accuracy: 0.8453
Epoch 3/5
626/626 [==============================] - 14s 23ms/step - loss: 0.1705 - accuracy: 0.9363 - val_loss: 0.4152 - val_accuracy: 0.8416
Epoch 4/5
626/626 [==============================] - 14s 22ms/step - loss: 0.1212 - accuracy: 0.9592 - val_loss: 0.4943 - val_accuracy: 0.8365
Epoch 5/5
626/626 [==============================] - 15s 23ms/step - loss: 0.0892 - accuracy: 0.9682 - val_loss: 0.4899 - val_accuracy: 0.8324


In [ ]:
y_pred=model.predict_classes(X_test)
y_pred

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

Performance Metrics and Accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3184,  551],
       [ 568, 2375]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8324348607367476

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      3735
           1       0.81      0.81      0.81      2943

    accuracy                           0.83      6678
   macro avg       0.83      0.83      0.83      6678
weighted avg       0.83      0.83      0.83      6678



In [ ]:
headline = 'Mom Warns Son to Watch Out for Idiots Rear‑Ends His Motorcycle'
# headline = 'Cows lose their jobs as milk prices drop'
# headline = 'Man Accused of Killing Lawyer Receives a New Attorney'
# headline = 'India to have over two billion vaccine doses during Aug-Dec'
# headline = '15 more patients die at Goa Medical College due to oxygen shortage'
# headline = 'City Union Bank donates Rs 1 crore to Relief Fund'
headline_len = len(headline)

In [ ]:
onehot=[one_hot(headline, headline_len)]
result = pad_sequences(onehot, padding='pre', maxlen=headline_len)

In [ ]:
final = model.predict(result)
if final > 0.5:
  print("Sarcastic")
else:
  print("Not Sarcastic")

Not Sarcastic


In [ ]:
import pickle
pickle.dump(model, open('sarcasm-detector.pkl', 'wb'))